## 2.2  Support Vector Machines Implementations
This task involves training a SVM classifier for each iris type, that is, $3$ SVMs for the $3$ distinct iris types.

This program implements SMO algorithm to solve SVM **without** using packages such as `sklearn` to earn some extra points, and works well on Q2.2.1 and Q2.2.2. However, since it didn't performed well on non-linear kernels, to avoid unnecessary points loss, I offer **both** codes using sklearn package and my class in Q2.2.3.

In [42]:
import numpy as np
import pandas as pd

# Set up basic parameters
np.random.seed(42)
np.set_printoptions(suppress=True, precision=3)

The definition of SVM class. It offers the same arguments/attributes/method as sklearn.svm.SVC, so that one can tranfer it into another easily.

In [43]:
class SVM:
	def __init__(self,
			C: float = 1.0,
			kernel: str = "linear",
			degree: int = 3,
			gamma: float = 0.5,
			coef0: float = 0.0,
			eps: float = 1e-3,
			max_iter: int = 1000
		):
		assert isinstance(C, (int, float)) and C > 0
		assert isinstance(kernel, str) and kernel in ("linear", "poly", "rbf", "sigmoid")
		assert isinstance(degree, int) and degree >= 1
		assert isinstance(gamma, (int, float)) and gamma > 0
		assert isinstance(eps, (int, float)) and eps > 0
		assert isinstance(max_iter, int) and max_iter >= 1
		self.C = C
		self.kernel = kernel
		self.degree = degree
		self.gamma = gamma
		self.coef0 = coef0
		self.eps = eps
		self.max_iter = max_iter

		self.coef_ = None # weights
		self.intercept_ = None # bias
		self.support_ = None # indices of support vectors

		self._X = None
		self._y = None
		self._alpha = None
		self._kernel_matrix = None
	
	def _linear_kernel(self, x1, x2):
		return x1 @ x2.T
	def _poly_kernel(self, x1, x2):
		return (self.gamma * x1 @ x2.T + self.coef0) ** self.degree
	def _rbf_kernel(self, x1, x2):
		if x1.ndim == 1 and x2.ndim == 1:
			return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)
		elif x1.ndim == 2 and x2.ndim == 2:
			return np.exp(-self.gamma * np.linalg.norm(x1[:, None] - x2[None, :], axis=-1) ** 2)
		else:
			return np.exp(-self.gamma * np.linalg.norm(x1 - x2, axis=-1) ** 2)
	def _sigmoid_kernel(self, x1, x2):
		return np.tanh(self.gamma * x1 @ x2.T + self.coef0)
	def _kernel(self, x1, x2):
		return {
			"linear": self._linear_kernel,
			"poly": self._poly_kernel,
			"rbf": self._rbf_kernel,
			"sigmoid": self._sigmoid_kernel
		}[self.kernel](x1, x2)
	
	def _objective(self, x):
		return x @ self.coef_ + self.intercept_
	def _objective_no_bias(self, x):
		return x @ self.coef_
	def _primal_weight(self):
		return (self._alpha * self._y) @ self._X
	def _set_primal_weight(self):
		self.coef_ = self._primal_weight()
	def _primal_bias(self):
		return np.mean(self._y[self.support_] - self._objective_no_bias(self._X[self.support_]))
	def _set_primal_bias(self):
		self.intercept_ = self._primal_bias()
	def decision_function(self, X):
		return self._objective(X)
	def _set_support(self):
		self.support_ = np.where((self._alpha > self.eps) & (self._alpha < self.C - self.eps))[0]

	
	# SMO algorithm
	def _smo_range(self, a1, a2, y1, y2, C) -> tuple:
		# calculate the range of alpha2 under constraints:
		# a1*y1 + a2*y2 = constant and 0 <= a1, a2 <= C
		l = max(0, a2 - a1) if y1 != y2 else max(0, a2 - (C - a1))
		r = min(C, a2 + (C - a1)) if y1 != y2 else min(C, a2 + a1)
		return l, r
	def _smo_pairwise(self, i, j) -> bool: # return True if alpha2 is changed
		a1, a2 = self._alpha[i], self._alpha[j]
		y1, y2 = self._y[i], self._y[j]
		l, r = self._smo_range(a1, a2, y1, y2, self.C)
		if l >= r:
			return False
		E1, E2 = self._objective(self._X[i]) - y1, self._objective(self._X[j]) - y2
		eta = self._kernel_matrix[i, i] + self._kernel_matrix[j, j] - 2 * self._kernel_matrix[i, j]
		if eta <= 0:
			return False
		b2 = np.clip(a2 + y2 * (E1 - E2) / eta, l, r)
		b1 = a1 + y1 * y2 * (a2 - b2)
		# print(b1.shape, b2.shape)
		if abs(b2 - a2) < self.eps * (b2 + a2 + self.eps):
			return False
		self._alpha[i], self._alpha[j] = b1, b2
		return True
	def _smo_main(self) -> bool:
		remaining = 50
		for t in range(self.max_iter):
			changed = False
			for i in range(self._X.shape[0]):
				j = np.random.randint(self._X.shape[0])
				while j == i: j = np.random.randint(self._X.shape[0])
				changed |= self._smo_pairwise(i, j)
				self._set_primal_weight()
				self._set_primal_bias()
			self._set_support()
			if not changed:
				remaining -= 1
				if remaining == 0:
					return True
			else:
				remaining = 50
		return False
	
	def fit(self, X, y):
		assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray)
		assert X.ndim == 2 and y.ndim == 1
		assert X.shape[0] == y.shape[0]
		self._X = X
		self._y = y
		self._alpha = np.zeros_like(y, dtype=np.float64)
		self._kernel_matrix = self._kernel(X, X)
		self.support_ = np.arange(X.shape[0])
		self._set_primal_weight()
		self._set_primal_bias()
		return self._smo_main()
	def predict(self, X):
		assert isinstance(X, np.ndarray)
		assert X.ndim == 2
		return np.sign(self.decision_function(X))
	def __call__(self, *args, **kwargs):
		return self.predict(*args, **kwargs)

In [44]:
# Load data / Preprocessing
iris_train_x, iris_train_y = pd.read_csv("x_train.csv").values, pd.read_csv("y_train.csv").values.reshape(-1)
iris_test_x, iris_test_y = pd.read_csv("x_test.csv").values, pd.read_csv("y_test.csv").values.reshape(-1)
for i in range(iris_train_x.shape[1]):
	iris_train_x[:, i] = (iris_train_x[:, i] - iris_train_x[:, i].mean()) / iris_train_x[:, i].std()
	iris_test_x[:, i] = (iris_test_x[:, i] - iris_test_x[:, i].mean()) / iris_test_x[:, i].std()

In [45]:
# Test functions
flower_names = ("setosa", "versicolor", "virginica")
def loss(y_pred, y_true):
	return (y_pred != y_true).astype(np.float32).mean()
def svm_test(group_id, group_name, model, soft_margin=True):
	iris_train_y_ = (iris_train_y == group_id).astype(np.float32) * 2 - 1
	iris_test_y_ = (iris_test_y == group_id).astype(np.float32) * 2 - 1
	model.fit(iris_train_x, iris_train_y_)
	train_pred, test_pred = model.predict(iris_train_x), model.predict(iris_test_x)
	train_loss, test_loss = loss(train_pred, iris_train_y_), loss(test_pred, iris_test_y_)
	print(f"{group_name} training error: {train_loss:.4f}, testing error: {test_loss:.4f}")
	if model.kernel == "linear":
		print(f"w_of_{group_name}: {model.coef_}, b_of_{group_name}: {model.intercept_}")
	support_indices = sorted(model.support_)
	print(f"support_vector_indices_of_{group_name}: {support_indices}")
	if soft_margin:
		slack = np.maximum(0, 1 - iris_train_y_[support_indices] * model.decision_function(iris_train_x[support_indices]))
		print(f"slack_variable_of_{group_name}: {slack}")

### 2.2.1 Calculation using Standard SVM Model (Linear Kernel) (With bonus)
Employ the standard SVM model with a linear kernel. Train your SVM on the provided training dataset and validate it on the testing dataset. Calculate the classification error for both the training and testing datasets, output the weight vector $w$, the bias $b$, and the indices of support vectors (start with $0$).

In [41]:
print("Q2.2.1 Calculation using Standard SVM Model:")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVM(C=1e5), soft_margin=False)

Q2.2.1 Calculation using Standard SVM Model:
setosa training error: 0.0000, testing error: 0.0000
w_of_setosa: [-0.304  0.288 -1.032 -0.819], b_of_setosa: -1.4092179485170044
support_vector_indices_of_setosa: [6, 28, 75]


KeyboardInterrupt: 

### 2.2.2 Calculate using SVM with Slack Variables (Linear Kernel) (With bonus)
For each $C = 0.2 \times t$, where $t = 1, 2, \ldots, 5$, train your SVM on the provided training dataset, and subsequently validate it on the testing dataset. Calculate the classification error for both the training and testing datasets, the weight vector $w$, the bias $b$, the indices of support vectors, and the slack variable $\xi$ (compute it as `abs(y-f(x))`)

In [ ]:
print("Q2.2.2 Calculate using SVM with Slack Variables (C = 0.2 x t, where t = 1, 2, ..., 5):")
for C in (0.2, 0.4, 0.6, 0.8, 1.0):
	print(f"C = {C}")
	for group_id, group_name in enumerate(flower_names):
		svm_test(group_id, group_name, SVM(C=C))
	print("-" * 50 if C < 1.0 else "")

Q2.2.2 Calculate using SVM with Slack Variables (C = 0.2 x t, where t = 1, 2, ..., 5):
C = 0.2
setosa training error: 0.0000, testing error: 0.0000
w_of_setosa: [-0.257  0.416 -0.635 -0.58 ], b_of_setosa: -0.8545149054556367
support_vector_indices_of_setosa: [23, 60, 66]
slack_variable_of_setosa: [0. 0. 0.]
versicolor training error: 0.2833, testing error: 0.2333
w_of_versicolor: [ 0.108 -0.955  0.138 -0.396], b_of_versicolor: -0.8786390544336532
support_vector_indices_of_versicolor: [8, 72, 81]
slack_variable_of_versicolor: [0. 0. 0.]
virginica training error: 0.0250, testing error: 0.0000
w_of_virginica: [ 0.028 -0.334  1.261  1.479], b_of_virginica: -1.9155197008647349
support_vector_indices_of_virginica: [43, 44, 61, 94]
slack_variable_of_virginica: [0.003 0.003 0.003 0.009]
--------------------------------------------------
C = 0.4
setosa training error: 0.0000, testing error: 0.0000
w_of_setosa: [-0.244  0.416 -0.734 -0.602], b_of_setosa: -0.8867263824849386
support_vector_indice

### 2.2.3 Calculate using SVM with Kernel Functions and Slack Variables (By sklearn, performing better)
Conduct experiments with different kernel functions for SVM, and set $C = 1$ for all cases. Calculate the classification error for both the training and testing datasets, the weight vector $w$, the bias $b$, the indices of support vectors for each kernel type:
* 2nd-order Polynomial Kernel
* 3rd-order Polynomial Kernel
* Radial Basis Function Kernel with $\sigma=1$
* Sigmoidal Kernel with $\sigma=1$

In [47]:
from sklearn.svm import SVC

print("Q2.2.3 Calculate using SVM with Kernel Functions and Slack Variables:")
print("(a) 2nd-order Polynomial Kernel:")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVC(C=1.0, kernel="poly", degree=2, gamma=1.0, coef0=1.0))
print("(b) 3rd-order Polynomial Kernel:")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVC(C=1.0, kernel="poly", degree=3, gamma=1.0, coef0=1.0))
print("(c) Radial Basis Function Kernel with \(\sigma = 1\):")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVC(C=1.0, kernel="rbf", gamma=0.5))
print("(d) Sigmoidal Kernel with \(\sigma = 1\):")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVC(C=1.0, kernel="sigmoid", gamma=0.5, coef0=1.0))

Q2.2.3 Calculate using SVM with Kernel Functions and Slack Variables:
(a) 2nd-order Polynomial Kernel:
setosa training error: 0.0000, testing error: 0.0000
support_vector_indices_of_setosa: [6, 28, 32, 75, 77, 90, 112]
slack_variable_of_setosa: [0.001 0.    0.    0.    0.    0.    0.   ]
versicolor training error: 0.0167, testing error: 0.0000
support_vector_indices_of_versicolor: [6, 16, 26, 53, 57, 58, 71, 74, 78, 85, 93, 97, 98, 100, 108, 113, 119]
slack_variable_of_versicolor: [0.    0.    0.    0.    0.992 1.761 0.616 0.76  0.029 0.    0.    0.431
 1.044 0.    0.    0.    0.595]
virginica training error: 0.0167, testing error: 0.0000
support_vector_indices_of_virginica: [22, 51, 57, 58, 71, 74, 78, 85, 93, 94, 97, 98, 100, 108, 113, 119]
slack_variable_of_virginica: [0.    0.    1.098 1.794 0.637 0.723 0.103 0.    0.    0.    0.414 0.936
 0.001 0.    0.    0.569]
(b) 3rd-order Polynomial Kernel:
setosa training error: 0.0000, testing error: 0.0000
support_vector_indices_of_setosa:

### 2.2.3 Calculate using SVM with Kernel Functions and Slack Variables (By myself, performing worse)

In [48]:
print("Q2.2.3 Calculate using SVM with Kernel Functions and Slack Variables:")
print("(a) 2nd-order Polynomial Kernel:")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVM(C=1.0, kernel="poly", degree=2, gamma=1.0, coef0=1.0))
print("(b) 3rd-order Polynomial Kernel:")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVM(C=1.0, kernel="poly", degree=3, gamma=1.0, coef0=1.0))
print("(c) Radial Basis Function Kernel with \(\sigma = 1\):")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVM(C=1.0, kernel="rbf", gamma=0.5))
print("(d) Sigmoidal Kernel with \(\sigma = 1\):")
for group_id, group_name in enumerate(flower_names):
	svm_test(group_id, group_name, SVM(C=1.0, kernel="sigmoid", gamma=0.5, coef0=1.0))

Q2.2.3 Calculate using SVM with Kernel Functions and Slack Variables:
(a) 2nd-order Polynomial Kernel:


e:\Anaconda\envs\torch2\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
e:\Anaconda\envs\torch2\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


setosa training error: 1.0000, testing error: 1.0000
support_vector_indices_of_setosa: []
slack_variable_of_setosa: []


KeyboardInterrupt: 

### Report
I actually implemented SVM with kernel tricks (in order to get all bonus points). Then I found that the performance was really bad under some kernels (such as sigmoidal kernel) thus I turn to use `sklearn` package.

However, the package still gives bad result using some kernels. It may be the fault of ther kernel itself.